In [ ]:
import pandas as pd 
import xgboost as xgb
import plotly.express as px
from sklearn.metrics import mean_absolute_error

In [ ]:
df_level = pd.read_csv('../datasets/Wood Brook/Loughborough-level-15min-Qualified.csv').fillna(0)
df_rain = pd.read_csv('../datasets/Wood Brook/Loughborough-University-rainfall-15min-Qualified.csv').fillna(0)

In [ ]:
df_level['value'] = pd.to_numeric(df_level['value'])

In [ ]:
df_merged = pd.merge(right=df_rain,left=df_level,left_on='dateTime',right_on='dateTime', how='inner')

In [ ]:
df_merged = df_merged.rename({'value_x':'level (m)','value_y': 'rainfall (mm)'},axis='columns')

In [ ]:
df_merged['dateTime'] = pd.to_datetime(df_merged['dateTime'])

In [ ]:
df_merged.set_index('dateTime',inplace=True)

In [ ]:
def create_rolling_sums(df,column,window_list):
    for window in window_list:
        df[f'{column}_sum_{window}'] = df[column].rolling(window).sum()
    return df

In [ ]:
windows = ['1h','2h','4h','8h','24h','7d','30d','90d']

In [ ]:
df_test_train = create_rolling_sums(df_merged,'rainfall (mm)',windows)
df_test_train = df_test_train.fillna(0)

In [ ]:
df_test_train.columns

In [ ]:
df_train =  df_test_train.loc['2017-05-01':'2023-12-31']
df_test = df_test_train.loc['2024-01-01':]


In [ ]:
x_train = df_train[
    [
        "rainfall (mm)_sum_1h",
        "rainfall (mm)_sum_2h",
        "rainfall (mm)_sum_4h",
        "rainfall (mm)_sum_8h",
        "rainfall (mm)_sum_24h",
        "rainfall (mm)_sum_7d",
        "rainfall (mm)_sum_30d",
        "rainfall (mm)_sum_90d",
    ]
]
x_test = df_test[
    [
        "rainfall (mm)_sum_1h",
        "rainfall (mm)_sum_2h",
        "rainfall (mm)_sum_4h",
        "rainfall (mm)_sum_8h",
        "rainfall (mm)_sum_24h",
        "rainfall (mm)_sum_7d",
        "rainfall (mm)_sum_30d",
        "rainfall (mm)_sum_90d",
    ]
]
y_train = df_train[["level (m)"]]

y_test = df_test[["level (m)"]]

In [ ]:
x_train

In [ ]:
x_train.dtypes

In [ ]:
y_train

In [ ]:
quantiles = [0.05, 0.50, 0.95]

In [ ]:
import sklearn
print(xgb.__version__)
print(sklearn.__version__)

In [ ]:
model = xgb.XGBRegressor(
max_depth= 15,
objective= 'reg:squarederror',
#quantile_alpha= 0.50,  # alpha is the quantile level
#eval_metric= 'rmse',
n_estimators= 10000, # Number of boosting rounds
verbosity= 2, # Verbose logging
#verbose= ,
device="cuda",
learning_rate= 0.001,
base_score = 1.5,
early_stopping_rounds = 50,

)
model.fit(x_train,y_train,eval_set=[(x_test, y_test)])
mean_preds = model.predict(x_test)



In [ ]:
model = xgb.XGBRegressor(
max_depth= 15,
objective= 'reg:tweedie',

#quantile_alpha= 0.50,  # alpha is the quantile level
#eval_metric= 'rmse',
n_estimators= 100000,
verbosity= 2, # Verbose logging
#verbose= ,
device="cuda",
learning_rate= 0.001,
base_score = 1,
early_stopping_rounds = 50,

)
model.fit(x_train,y_train,eval_set=[(x_test, y_test)])
tweedle_preds = model.predict(x_test)


In [ ]:
predictions ={}
predictions['mean pred'] = mean_preds 
predictions['tweedle pred'] = tweedle_preds

In [ ]:
models = {}
#predictions = {f"{q*100}th_centile": [] for q in quantiles}

for q in quantiles:
    # Create DMatrix for XGBoost
    # Set parameters
    # params = {
        
    # #     'max_depth': 25,
    # #     'objective': 'reg:quantileerror',
    # #     'quantile_alpha': q,  # alpha is the quantile level
    # #     'eval_metric': 'mae',
    # #     'n_estimators': 10000, # Number of boosting rounds
    # #     'verbosity': 2, # Verbose logging
    # }

    # Train the model
    model = xgb.XGBRegressor(
        max_depth= 5,
        objective= 'reg:quantileerror',
        quantile_alpha= q,  # alpha is the quantile level
        #eval_metric= 'rmse',
        n_estimators= 100000, # Number of boosting rounds
         # Verbose logging
        device="cuda",
        learning_rate= 0.005,
        base_score = 1,
        early_stopping_rounds = 200
    )
    
    model.fit(x_train,y_train,eval_set=[(x_test, y_test)])
    #print(model.evals_result())
    models[q] = model

    # Make predictions
    preds = model.predict(x_test)
    predictions[f"{q*100}th_centile"] = preds

# Convert predictions to a DataFrame for better visualization
quantile_predictions = pd.DataFrame(predictions)


In [ ]:
quantile_predictions

In [ ]:
y_test.reset_index(inplace=True)
y_test_plus_pred = pd.concat([y_test, quantile_predictions,], axis=1)
y_test.set_index('dateTime')
y_test_plus_pred

In [ ]:
y_test_plus_pred.columns

In [ ]:
fig = px.line (y_test_plus_pred, x='dateTime',y=['level (m)', 'mean pred', 'tweedle pred', '5.0th_centile',
       '50.0th_centile', '95.0th_centile'], title="Wood Brook Level Predictions")
fig.add_hline(y=0.7)
fig.show()

In [ ]:
df_weather =pd.read_csv('../datasets/Wood Brook/suttonboningtondata.CSV')
df_weather

In [ ]:
df_test_train['month'] = df_test_train.index.month
df_test_train['year'] = df_test_train.index.year
df_test_train

In [ ]:
df_train_date =  df_test_train.loc['2017-05-01':'2023-12-31']
df_test_date = df_test_train.loc['2024-01-01':]


In [ ]:
x_train = df_train_date[
    [
        "rainfall (mm)_sum_1h",
        "rainfall (mm)_sum_2h",
        "rainfall (mm)_sum_4h",
        "rainfall (mm)_sum_8h",
        "rainfall (mm)_sum_24h",
        "rainfall (mm)_sum_7d",
        "rainfall (mm)_sum_30d",
        "rainfall (mm)_sum_90d",
        'month',
        'year',
    ]
]
x_test = df_test_date[
    [
        "rainfall (mm)_sum_1h",
        "rainfall (mm)_sum_2h",
        "rainfall (mm)_sum_4h",
        "rainfall (mm)_sum_8h",
        "rainfall (mm)_sum_24h",
        "rainfall (mm)_sum_7d",
        "rainfall (mm)_sum_30d",
        "rainfall (mm)_sum_90d",
        'month',
        'year',
    ]
]
y_train = df_train[["level (m)"]]

y_test = df_test[["level (m)"]]

In [ ]:
predictions ={}


In [ ]:
models_date = {}
#predictions = {f"{q*100}th_centile": [] for q in quantiles}

for q in quantiles:
    # Create DMatrix for XGBoost
    # Set parameters
    # params = {
        
    # #     'max_depth': 25,
    # #     'objective': 'reg:quantileerror',
    # #     'quantile_alpha': q,  # alpha is the quantile level
    # #     'eval_metric': 'mae',
    # #     'n_estimators': 10000, # Number of boosting rounds
    # #     'verbosity': 2, # Verbose logging
    # }

    # Train the model
    model = xgb.XGBRegressor(
        max_depth= 15,
        objective= 'reg:quantileerror',
        quantile_alpha= q,  # alpha is the quantile level
        #eval_metric= 'rmse',
        n_estimators= 100000, # Number of boosting rounds
         # Verbose logging
        device="cuda",
        learning_rate= 0.005,
        base_score = 1,
        early_stopping_rounds = 200
    )
    
    model.fit(x_train,y_train,eval_set=[(x_test, y_test)])
    #print(model.evals_result())
    models[q] = model

    # Make predictions
    preds = model.predict(x_test)
    predictions[f"{q*100}th_centile"] = preds

# Convert predictions to a DataFrame for better visualization
quantile_predictions_month = pd.DataFrame(predictions)

In [ ]:
y_test.reset_index(inplace=True)
x_test.reset_index(inplace=True)
y_test_plus_pred_date = pd.concat([y_test, quantile_predictions_month,], axis=1)
y_test_plus_pred_date =pd.merge(y_test_plus_pred_date,x_test,on='dateTime')

y_test.set_index('dateTime')
y_test_plus_pred_date

In [ ]:
fig = px.line (y_test_plus_pred_date, x='dateTime',y=['level (m)', '5.0th_centile',
       '50.0th_centile', '95.0th_centile'], title="Wood Brook Level Predictions")
fig.add_hline(y=0.7)

fig.show()

In [ ]:
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.Figure()

fig.update_layout( yaxis=dict( title='Primary Y-Axis' ), yaxis2=dict( title='Rainfall (mm)', overlaying='y', side='right' ) ) 

#
fig.add_trace(go.Bar(
    x=y_test_plus_pred_date['dateTime'], y=y_test_plus_pred_date['rainfall (mm)_sum_1h'], yaxis='y2', name = 'rainfall'
))

# Add 'bottom' series
fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], y=y_test_plus_pred_date['5.0th_centile'],
    mode='lines',
    line=dict(width=0.5, color='rgba(0, 100, 80)'),
    name='5th percentile'
))
# Add 'upper' series and fill area between 'bottom' and 'upper'
fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], y=y_test_plus_pred_date['95.0th_centile'],
    mode='lines',
    line=dict(width=0.5, color='rgba(0, 100, 80, 0.2)'),
    fill='tonexty',  # Fill area between 'bottom' and 'upper'
    name='95th Percentile'
))




# Add 'middle' series
fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], y=y_test_plus_pred_date['50.0th_centile'],
    mode='lines',
    line=dict(width=2, color='rgb(0, 0, 255)'),
    name='predicted level'
))

fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], y=y_test_plus_pred_date['level (m)'],
    mode='lines',
    line=dict(width=2, color='rgb(170, 20,40)'),
    name='observed level'
))
fig.add_hline(0.7,line_dash="dot", annotation_text="Flooding Possible", annotation_position="top right")

# Update layout to flip the secondary y-axis

# Show the plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Assuming your data is in pandas dataframe `y_test_plus_pred_date`

# Create the figure
fig = go.Figure()

# Add the Rainfall trace (flipped, with opacity 0.7)
fig.add_trace(go.Bar(
    x=y_test_plus_pred_date['dateTime'], 
    y=-y_test_plus_pred_date['rainfall (mm)_sum_1h'],  # Flip by making the y-values negative
    name='Rainfall',
    opacity=0.7,  # Set transparency of the bars
    marker=dict(color='rgba(0, 100, 255, 0.7)'),  # Optional: Set the color with alpha
    xaxis='x2'  # Link this trace to the secondary x-axis (top x-axis)
))

# Add the remaining traces (percentiles and levels) on the primary x-axis
fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], 
    y=y_test_plus_pred_date['5.0th_centile'],
    mode='lines',
    line=dict(width=0.5, color='rgba(0, 100, 80)'),
    name='5th percentile'
))

fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], 
    y=y_test_plus_pred_date['95.0th_centile'],
    mode='lines',
    line=dict(width=0.5, color='rgba(0, 100, 80, 0.2)'),
    fill='tonexty',  # Fill area between 'bottom' and 'upper'
    name='95th Percentile'
))

fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], 
    y=y_test_plus_pred_date['50.0th_centile'],
    mode='lines',
    line=dict(width=2, color='rgb(0, 0, 255)'),
    name='Predicted level'
))

fig.add_trace(go.Scatter(
    x=y_test_plus_pred_date['dateTime'], 
    y=y_test_plus_pred_date['level (m)'],
    mode='lines',
    line=dict(width=2, color='rgb(170, 20, 40)'),
    name='Observed level'
))

# Add a horizontal line to the plot (Flooding Possible)
fig.add_hline(
    y=0.7, 
    line_dash="dot", 
    annotation_text="Flooding Possible", 
    annotation_position="top right"
)

# Update layout to make sure the plot looks good
fig.update_layout(
    title="Rainfall and Water Levels",
    yaxis=dict(title='Rainfall (mm) and Water Levels (m)', side='left'),
    xaxis=dict(title="Date/Time", showgrid=True),  # Primary x-axis (bottom)
    xaxis2=dict(  # Secondary x-axis (top for rainfall)
        title="Date/Time",
        overlaying="x",  # Link to primary x-axis
        side="top",  # Position at the top of the plot
        showgrid=False,  # Disable grid lines on the secondary x-axis
        tickvals=y_test_plus_pred_date['dateTime'],  # Ensure the ticks align with the primary x-axis
    ),
    showlegend=True,
    height=600,  # Adjust the height if needed
    bargap=0.1  # Add some gap between bars for better visualization
)

# Show the plot
fig.write_html('plot.html')

